In [1]:
import pandas as pd

In [10]:
df = pd.read_csv('stanweb.dat', sep='\s+', names=['a', 'b', 'value'])

In [11]:
df

,a,b,value
0,1,6548,0.500000
1,1,15409,0.500000
2,2,252915,0.032258
3,2,246897,0.032258
4,2,251658,0.032258
...,...,...,...
2382907,281903,216688,0.142857
2382908,281903,90591,0.142857
2382909,281903,94440,0.142857
2382910,281903,56088,0.142857
